In [2]:
import os
os.makedirs('output', exist_ok=True)

import sys
sys.path.append('../')

from utils import load_walksheds, overlay_wks
import pandas as pd
import numpy as np
import geopandas as gpd
import os

os.makedirs('output', exist_ok=True)

* walskhedbuffer is the Metro station walksheds with overlapping boundaries
* all_boundaries All boundaries for census blocks in MD, DC and VA
* shf is the datafame for the [senior housing facility](https://nhpd.preservationdatabase.org/Data)
* from shf obtain data pertaining to MD, DC, and VA respectively.
* Extract relevant columns of interests

In [3]:
wksp5, wksp75 = load_walksheds()

all_boundaries= gpd.read_file('../../Data/Merge Boundaries/MD_DC_VA/Block MD_DC_VA.shp')
shf = pd.read_excel('../../Data/Active Properties.xlsx')

In [4]:
shf = shf[(shf.State=='MD') | (shf.State=='DC') | (shf.State=='VA')].reset_index(drop=True)

In [5]:
shf = shf[['PropertyName', 'PropertyAddress', 'CountyCode', 'CensusTract', 'TotalUnits']].dropna()
shf['CountyCode'] = shf['CountyCode'].astype(np.int64)
shf['CensusTract'] = shf['CensusTract'].astype(np.int64).astype('str')
shf.head()

,PropertyName,PropertyAddress,CountyCode,CensusTract,TotalUnits
0,ELDRIDGE BARSTOW,3920 Hallowing Point Rd,24009,24009860703,55
1,HOMES OF GREATER BALTIMORE,429 N Eutaw St,24510,24510040100,58
2,HMDE VILLAGE,113 Mitchell St,24041,24041960700,80
3,SKYLINE APARTMENTS,2549 Elvans Rd SE,11001,11001007406,398
4,GREENWILLOW MANOR,610 Pennsylvania Ave,24510,24510170100,198


* join shf to all_boundaries dc, md, va
* align walkshed and joined data to the same CRS
* drop duplicates
* Get all the total units of senior housing facilities within the walksheds
* save result to CSV

In [6]:
all_boundaries['GEOID200'] = all_boundaries['STATEFP20'] + all_boundaries['COUNTYFP20'] + all_boundaries['TRACTCE20']

In [7]:
join_shf_boundary = all_boundaries.merge(shf, left_on='GEOID200', right_on='CensusTract', how='inner')

In [8]:
join_shf_boundary = join_shf_boundary.to_crs('EPSG:4326')

In [9]:
join_shf_boundary = join_shf_boundary.drop_duplicates(subset="PropertyAddress")

In [10]:
shf_intersect_p5, shf_intersect_p75 = overlay_wks(join_shf_boundary)

In [11]:
shf_intersect_p5

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,...,PropertyAddress,CountyCode,CensusTract,TotalUnits,Name_1,Acres,Shape_Leng_2,Shape_Area_2,StnCode,geometry
0,24,031,704404,1004,240317044041004,Block 1004,224514.0,415.0,+39.0066882,-077.0935600,...,3 Pooks Hill Rd,24031,24031704404,189,MEDICAL CENTER,379.404828,19251.879962,1.652681e+07,None,"POLYGON ((-77.09737 39.00454, -77.09740 39.004..."
1,24,033,803514,3003,240338035143003,Block 3003,764432.0,1568.0,+38.9010034,-076.8441720,...,500 Harry S Truman Dr N,24033,24033803514,110,LARGO TOWN CENTER,229.147776,14888.085213,9.981637e+06,None,"MULTIPOLYGON (((-76.84639 38.90545, -76.84602 ..."
2,24,033,803514,3003,240338035143003,Block 3003,764432.0,1568.0,+38.9010034,-076.8441720,...,520 Largo Center Dr,24033,24033803514,100,LARGO TOWN CENTER,229.147776,14888.085213,9.981637e+06,None,"MULTIPOLYGON (((-76.84639 38.90545, -76.84602 ..."
3,24,031,701215,1001,240317012151001,Block 1001,75718.0,8551.0,+39.0293302,-077.1062257,...,5707 Luxemburg St,24031,24031701215,131,GROSVENOR-STRATHMORE,303.315667,20962.609643,1.321238e+07,None,"POLYGON ((-77.10360 39.02437, -77.10350 39.023..."
4,24,031,704401,1002,240317044011002,Block 1002,131775.0,1500.0,+39.0185393,-077.0996437,...,6040 Southport Dr,24031,24031704401,18,GROSVENOR-STRATHMORE,303.315667,20962.609643,1.321238e+07,None,"POLYGON ((-77.10279 39.02204, -77.10281 39.022..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,51,013,102002,1004,510131020021004,Block 1004,19161.0,0.0,+38.8793137,-077.1050422,...,3825 7th St N,51013,51013102002,103,VIRGINIA SQUARE-GMU,327.112346,15843.360939,1.424896e+07,None,"POLYGON ((-77.10396 38.87956, -77.10393 38.879..."
297,51,013,101804,2003,510131018042003,Block 2003,12952.0,0.0,+38.8887278,-077.0889253,...,2230 Fairfax Dr,51013,51013101804,108,CLARENDON,393.059326,17169.246622,1.712160e+07,None,"POLYGON ((-77.08848 38.88891, -77.08841 38.888..."
298,51,013,101804,2003,510131018042003,Block 2003,12952.0,0.0,+38.8887278,-077.0889253,...,2399 11TH ST N,51013,51013101804,364,CLARENDON,393.059326,17169.246622,1.712160e+07,None,"POLYGON ((-77.08848 38.88891, -77.08841 38.888..."
299,51,013,103505,1001,510131035051001,Block 1001,129626.0,0.0,+38.8630474,-077.0606448,...,1500 S Fern St Apt 626,51013,51013103505,302,PENTAGON,251.708533,15685.733729,1.096438e+07,None,"POLYGON ((-77.05863 38.86513, -77.05868 38.864..."


In [12]:
shf_intersect_p5 = shf_intersect_p5[['Name_1', 'TotalUnits']].copy()
shf_intersect_p75 = shf_intersect_p75[['Name_1', 'TotalUnits']].copy()


In [13]:
shf_intersect_p5.groupby('Name_1').sum().reset_index().rename(columns={'TotalUnits':'TotalUnits_p5'}).to_csv('output/senior_housing_facilitiesDMV_p5.csv')
shf_intersect_p75.groupby('Name_1').sum().reset_index().rename(columns={'TotalUnits':'TotalUnits_p75'}).to_csv('output/senior_housing_facilitiesDMV_p75.csv')
